# 스킬 예시 

이 노트북은 우리 모델의 다양한 기능과 스킬을 강조하기 위한 것입니다. 이것은 완전한 목록이 아니며, 예시들은 무슨 일이 일어나고 있는지 쉽게 이해할 수 있도록 간단하게 구성되어 있습니다. 이러한 이유로 대부분의 예시에서 Lite를 사용하고 있습니다.

모델을 호출하는 방법과 지원되는 기능에 대한 세부 사항은 다루지 않을 것입니다. 이에 대해서는 Quick Start 노트북을 참조하세요.

---

## 설정

먼저 Quick Start에 설명된 단계를 따랐는지 확인하세요. 특히, 계정과 지역에서 모델이 활성화되어 있는지 확인하세요

**참고**: _아래 예시들은 텍스트 이해 사용 사례이며 Micro, Lite 또는 Pro 모델과 함께 사용할 수 있습니다. `model_id`를 변경하여 다른 모델을 시도해볼 수 있습니다_

In [5]:
import boto3
import json

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

DEFAULT_MODEL_ID = PRO_MODEL_ID

client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
)


def invoke(prompt, model_id=DEFAULT_MODEL_ID):
    messages = [
        {"role": "user", "content": [{"text": prompt}]},
    ]

    inference_config = {
        "max_new_tokens": 2048,
        "top_p": 0.9,
        "top_k": 20,
        "temperature": 0.7,
    }

    request = {
        "messages": messages,
        "inferenceConfig": inference_config,
    }

    response = client.invoke_model(modelId=model_id, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response["output"]["message"]["content"][0]["text"]

---

## 스킬 예시

### 텍스트 요약

이 예시에서는 루이스 캐럴의 이상한 나라의 앨리스 전체 책을 다운로드하고, 모델에게 이를 요약해달라고 요청합니다.

In [6]:
import urllib.request


def download_url(url):
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read().decode("utf-8")
            return content
    except urllib.error.URLError as e:
        print(f"Error downloading URL: {e.reason}")
        return None


book_content = download_url("https://www.gutenberg.org/files/11/11-0.txt")
print(f"{book_content[:100]=}")
print(f"{len(book_content)=}")
print(invoke(f"Summarize the following book in Korean, less than 500 words: {book_content}"))

book_content[:100]='*** START OF THE PROJECT GUTENBERG EBOOK 11 ***\r\n[Illustration]\r\n\r\n\r\n\r\n\r\nAlice’s Adventures in Wonde'
len(book_content)=148078
앨리스의 원더랜드 모험

작가: 루이스 캐롤

앨리스는 자매와 함께 강둑에 앉아 있다가 지나가는 흰 토끼를 따라 토끼 구멍으로 떨어지며 원더랜드 모험을 시작합니다. 흰 토끼는 시계를 가지고 있었고, 앨리스는 호기심에 뒤따라갑니다. 구멍 속으로 떨어진 앨리스는 몇 번이나 크기가 변하며 다양한 생물들을 만나게 됩니다.

앨리스는 먼저 쥐와 만나 자신의 크기가 변하는 것에 대해 이야기하고, 흰 토끼의 집에 들어가 여러 가지 물건을 만지다가 또 다른 크기 변화를 경험합니다. 그녀는 뒤죽박죽 정원 파티에 참석하여 모자장수와 3월의 토끼를 만나고, 거짓가메와 함께 수영을 하며 독특한 춤을 배웁니다. 앨리스는 또한 공작 여왕을 만나 크로켓 게임에 참여하고, 목이 잘리지 않을까 걱정하며 재판에 참석하기도 합니다.

앨리스는 자신의 크기가 변하는 것에 대해 혼란스러워하지만, 점차 이 세계에 적응하게 됩니다. 그녀는 원더랜드에서 만난 생물들과 대화를 나누며 자신의 정체성에 대해 고민하고, 독특한 경험을 통해 성장하는 모습을 보여줍니다. 결국 앨리스는 원더랜드에서의 모험이 꿈이었음을 깨닫고, 현실로 돌아오게 됩니다.

이 책은 상상력과 호기심을 자극하는 동시에, 자아 정체성과 성장에 대한 주제를 다루고 있습니다. 앨리스의 모험은 독자들에게 다양한 해석의 가능성을 제시하며, 판타지 문학의 고전으로 자리 잡고 있습니다.


### 콘텐츠 생성

이 예시는 모델이 창의적인 텍스트를 생성할 수 있는 방법을 보여줍니다

In [ ]:
print(invoke(f"Write an ode to the commuting to work"))

### 질문 답변

이 예시는 모델이 문맥 내 콘텐츠에 대한 질문에 어떻게 응답하는지 보여줍니다.

In [8]:
manual = """
오늘부터 세탁이 그 어느 때보다 쉬워집니다. 아미카 세탁기는 뛰어난 사용 편의성과 탁월한 효과를 결합했습니다.
설명서를 읽고 나면 세탁기 작동에 어려움이 없을 것입니다. 포장 및 공장 출하 전, 이 세탁기의 안전성과 기능이 철저히 테스트되었습니다.
이러한 테스트로 인해 기기 내부에 약간의 습기나 물 흔적이 남아있을 수 있습니다.
제품 전원을 켜기 전에 사용설명서를 주의 깊게 읽어주시기 바랍니다. 본 설명서의 지침을 따르면 잘못된 사용으로부터 보호받을 수 있습니다.
이 사용설명서를 가까운 곳에 보관하십시오. 불의의 사고를 방지하기 위해 지침을 주의 깊게 따라야 합니다.
중요! 이 세탁기는 세제 bath에서 기계 세탁이 가능하도록 설계된 직물과 의류를 세탁하기 위한 순수 가정용입니다.
세탁기를 주 전원에 연결하기 전에 안전 포장을 제거하고 기계가 수평인지 확인하십시오.
제조업체는 제품 작동에 영향을 미치지 않는 변경을 할 권리가 있습니다.

사용 방법
기계는 아래 지침을 주의 깊게 읽은 후에만 사용해야 합니다.
기계를 야외나 온도가 영하로 내려갈 수 있는 곳에서 절대 사용하지 마십시오.
기계는 어린이 또는 지침을 읽지 않은 사람이 작동해서는 안 됩니다.
중요! 0°C 이하의 온도는 세탁기를 손상시킬 수 있습니다!
기계가 영하 온도에서 보관되거나 운송된 경우, 사용하기 전에 영상 온도에서 8시간 동안 적응시켜야 합니다.

제조업체 선언 제조업체는 본 제품이 다음 유럽 지침의 요건을 충족함을 선언합니다:
저전압 지침 2014/35/UE,
전자기 호환성(EMC) 지침 2004/30/UE,
ErP 지침 2009/125/EC,
RoHS 지침 2011/65/UE, 이에 따라 심볼로 표시되었으며 시장 규제기관에 제공된 적합성 선언이 발급되었습니다.
안전 참고 사항
자동 세탁기용으로 설계된 세제와 첨가제만 사용하십시오, 특히 면직물 프로그램 사용 시 그러합니다. 용제를 함유한 제품은 사용하지 마십시오. 이는 기계의 특정 부품을 손상시키거나 유독 가스를 방출할 수 있습니다. 또한 발화 및 폭발 위험이 있을 수 있습니다.
원치 않는 물체가 기기 안에 들어가지 않도록 하십시오. 세탁할 의류의 모든 주머니를 완전히 비우고 단추와 지퍼를 고정하십시오. 작고 느슨한 물건은 세탁망 또는 베개 케이스 안에 넣으십시오. 이러한 물건들은 파손되거나 드럼이나 내부 탱크를 손상시킬 수 있습니다. 권장되는 최대 세탁물 용량을 초과하지 마십시오.
제어판이 물에 젖지 않도록 보호하십시오. 세탁한 옷을 기계 위에 올려놓지 마십시오!
운송을 위해 기계를 준비하기 전에 전원을 뽑으십시오. 드럼 잠금 볼트가 제자리에 있지 않으면 세탁기를 운반하지 마십시오. '기계 설치' 장을 참조하십시오. 운반 시 기기를 수직 위치로 유지하십시오.
세탁기에 관한 기본 정보를 표시하는 데이터 플레이트는 후면 케이싱 상단부에 위치해 있습니다.
세탁기 작동 중 도어 유리는 뜨겁습니다. 어린이가 기기 근처에 가까이 오지 않도록 하십시오.
"""

print(
    invoke(f"""
**Question:**
사용해도 되는 온도는 어떻게 되나요?
주의 - 텍스트에 없는 정보는 사용하지 마세요!

**Reference Text:**
{manual}
"""
    )
)


**Answer:**

기계를 야외나 온도가 영하로 내려갈 수 있는 곳에서 절대 사용하지 마십시오. 중요! 0°C 이하의 온도는 세탁기를 손상시킬 수 있습니다! 따라서 사용 가능한 온도는 0°C 이상이어야 합니다. 기계가 영하 온도에서 보관되거나 운송된 경우, 사용하기 전에 영상 온도에서 8시간 동안 적응시켜야 합니다.


### 텍스트 번역

아래는 앙투안 드 생텍쥐페리의 어린 왕자에서 발췌한 구절입니다. 우리는 모델에게 언어를 식별하고, (영어로) 책을 식별하도록 요청하고, 내러티브에 대해 질문하고, 마지막으로 영어로 번역해달라고 요청할 것입니다.

In [11]:
text = "J'ai sauté sur mes pieds comme si j'avais été frappé par la foudre. J'ai bien frotté mes yeux. J'ai bien regardé. Et j'ai vu un petit bonhomme tout à fait extraordinaire qui me considérait gravement. Voilà le meilleur portrait que, plus tard, j'ai réussi à faire de lui. Mais mon dessin, bien sûr, est beaucoup moins ravissant que le modèle. Ce n'est pas ma faute. J'avais été découragé dans ma carrière de peintre par les grandes personnes, à l'age de six ans, et je n'avais rien appris à dessiner, sauf les boas fermés et les boas ouverts. "
print(invoke(f"무슨 언어로 되어 있나요?: {text}"))
print("--------")
print(invoke(f"어떤 책에서 발췌한 내용인가요?: {text}"))
print("--------")
print(invoke(f"캐릭터가 본 것은 무엇인가요?: {text}"))
print("--------")
print(invoke(f"한국어로 번역해주세요: {text}"))

이 텍스트는 프랑스어로 되어 있습니다. 

프랑스어는 로맨스 언어군에 속하며, 주로 프랑스, 벨기에, 스위스, 캐나다(퀘벡 등), 아프리카의 일부 국가 등에서 사용됩니다. 

이 텍스트에서는 화자가 어떤 놀라운 작은 인물을 보게 되고, 그 인물을 그리려고 했지만 어렸을 때 그림을 그리는 것을 막혔기 때문에 제대로 그릴 수 없었다는 내용을 담고 있습니다. 여기서 언급된 "boas fermés et les boas ouverts"는 작은 아이들이 종종 판타지적인 그림을 그릴 때 나오는 상상의 생물이나 모양을 의미할 수 있습니다.
--------
이 발췌 내용은 앙투안 드 생텍쥐페리의 《어린 왕자》(Le Petit Prince)에서 나온 것입니다. 이 책은 1943년에 발표된 소설로, 어린 왕자가 다양한 행성을 여행하며 만나는 사람들과의 이야기를 통해 깊은 철학적 메시지를 전달합니다. 

이 구절에서 어린 왕자를 만난 조종사(나레이터)는 갑자기 나타난 작고 특별한 인물(어린 왕자)에 놀라고, 자신의 그림 실력이 부족하다는 것을 안타까워하는 내용을 담고 있습니다. 어린 시절 어른들의 말 때문에 화가의 꿈을 포기했다는 아쉬움도 함께 전달합니다.
--------
캐릭터가 본 것은 "tout à fait extraordinaire" (아주 특별한) 작은 사람, 즉 작은 인물입니다. 이 작은 인물은 캐릭터를 진지하게 바라보고 있었습니다. 캐릭터는 이 장면을 나중에 그림으로 그려보았지만, 실제로 본 모델만큼 매력적이지는 않았습니다. 캐릭터가 어렸을 때 어른들에 의해 화가의 꿈을 포기하게 되었고, 그 이후로 제대로 된 그림 공부를 하지 않았기 때문에, 그의 그림 실력이 부족하다는 것을 설명하고 있습니다. 

이 문단은 앙투안 드 생텍쥐페리의 《어린 왕자》에서 따온 것으로 보입니다. 캐릭터는 조종사인 나레이터이고, 그가 본 작은 인물은 바로 어린 왕자입니다.
--------


ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again.

### 텍스트 분석 및 추천

이 예시에서는 텍스트와 감정을 이해하는 모델의 능력을 보여줍니다

In [15]:
review = """우선, 가격만으로도 아마 당신이 품질에 대한 리뷰를 찾아보게 되었을 것입니다. 걱정하지 마세요, 가격이 암시하는 것보다 훨씬 더 많은 가치가 있는 가방입니다. 박스에서 꺼내자마자, 가죽 냄새가 거의 없는 부드러운 가죽이 느껴집니다. 중간 갈색 색상은 클래식하고 다양한 스타일과 잘 어울립니다. 전체적으로, 잘 만들어졌으며 여행 중 수납을 위한 많은 공간을 제공합니다.

사용할수록 가죽 향이 증가하는 것을 알게 되었는데, 이는 꽤 기분 좋은 느낌입니다. 만져보면 여전히 부드럽고 내구성도 상당히 좋아 보입니다.

전반적으로, 알려진 신뢰할 수 있는 브랜드에서 나온 가죽 가방을 100달러 미만에 구할 수 있다는 점을 고려하면 가격 대비 훌륭한 가방입니다.

이 가방에서 찾은 유일한 단점은 가방을 열고 닫는 데 사용되는 걸쇠입니다. 보기에는 좋고 기능도 하지만, 약간 저렴하고 정확히 빠르고 쉽게 사용할 수 있는 것은 아닙니다. 제자리에 잘 들어맞지만, 클립을 풀기 위한 스프링에 탄력이 많이 없어서 여는 것이 어려운 도전입니다. 저는 이에 익숙해졌지만, 순간적으로 명함을 빠르게 꺼내려 한다면 그 귀찮은 클립과 씨름하느라 내일까지 서 있게 될 수도 있습니다. 사용자마다 다를 수 있는 약간의 과장이지만, 그럼에도 언급할 가치가 있는 부분입니다.

이 구매에 대해 너무 깊게 생각하지 마세요. 500달러짜리 가방에 예산이 있다면, 물론 계속 검색하세요. 당신이 지불한 만큼의 가치를 받는다고 믿기 때문입니다. 하지만, 제조사 소매가격은 300달러이므로, 하위 등급의 제품은 아닙니다. 기회가 있을 때 이 가방을 구매하세요!"""
print(
    invoke(
        f"리뷰의 내용을 보고, 긍정적인지 부정적인지 판단해보세요: {review}"
    )
)
print("---")
print(invoke(f"이 제품 리뷰에서 좋아하는 점들을 나열하세요: {review}"))

이 리뷰는 대체로 긍정적입니다. 리뷰어는 가방의 다음과 같은 여러 측면을 긍정적으로 평가합니다:

1. **가격 대비 가치**: 리뷰어는 가격이 암시하는 것보다 훨씬 더 많은 가치가 있다고 말합니다.
2. **품질**: 부드러운 가죽과 잘 만들어진 구조를 칭찬합니다.
3. **디자인**: 중간 갈색 색상이 클래식하고 다양한 스타일과 잘 어울린다고 언급합니다.
4. **수납 공간**: 여행 중에 많은 수납 공간을 제공한다고 합니다.
5. **내구성**: 만져보면 여전히 부드럽고 내구성도 상당히 좋다고 평가합니다.

그러나, 리뷰어는 가방의 걸쇠에 대해 약간의 불만을 언급합니다:

1. **걸쇠의 품질**: 걸쇠가 약간 저렴하게 느껴지고, 클립을 풀기 위한 스프링에 탄력이 부족하여 여는 데 어려움이 있다고 언급합니다.

전반적으로, 이 리뷰는 가방의 대부분의 측면에 대해 긍정적이며, 단 하나의 단점을 제외하고는 전반적으로 만족스러운 구매 경험을 전달합니다.
---
이 제품 리뷰에서 좋아하는 점들을 나열하면 다음과 같습니다:

1. **가격 대비 가치**:
   - 저렴한 가격에도 불구하고 훨씬 더 많은 가치를 제공합니다.
   - 100달러 미만에 알려진 신뢰할 수 있는 브랜드의 가죽 가방을 구할 수 있다는 점.

2. **품질**:
   - 부드러운 가죽 소재로 거의 가죽 냄새가 없음.
   - 사용할수록 가죽 향이 증가하며, 이는 기분 좋은 느낌을 줌.
   - 만져보면 여전히 부드럽고 내구성도 상당히 좋음.

3. **디자인**:
   - 중간 갈색 색상이 클래식하고 다양한 스타일과 잘 어울림.

4. **실용성**:
   - 여행 중 수납을 위한 많은 공간을 제공함.
   - 잘 만들어져 있음.

5. **전반적인 만족도**:
   - 제조사 소매가격이 300달러임에도 불구하고, 하위 등급의 제품이 아님.
   - 지불한 만큼의 가치를 받는다고 믿음.


### 대화 및 대화형 상호작용

다음 예시는 메시지를 사용하여 주고받는 대화를 만드는 방법을 보여줍니다.

In [17]:
multi_turn_messages = []


def turn(prompt):
    multi_turn_messages.append({"role": "user", "content": [{"text": prompt}]})

    inference_config = {
        "max_new_tokens": 300,
        "top_p": 0.9,
        "top_k": 20,
        "temperature": 0.7,
    }

    request = {
        "messages": multi_turn_messages,
        "system": [{ "text": "짧고 간결한 내용으로 답변하세요." }],
        "inferenceConfig": inference_config,
    }

    response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    output_message = model_response["output"]["message"]
    multi_turn_messages.append(output_message)
    assistant_resp = output_message["content"][0]["text"]
    print(f"User: {prompt}")
    print(f"Assistant: {assistant_resp}")
    print("")


turn("나는 앨리스고, 30살이야. 9년 전에 시애틀로 이사왔어.")
turn("내 이름이 뭐라고 했지?")
turn("내가 어디 살고 있게?")
turn("내가 몇 살 때 이사왔게?")

User: 나는 앨리스고, 30살이야. 9년 전에 시애틀로 이사왔어.
Assistant: 알겠어, 앨리스. 30살이고 9년 전 시애틀로 이사왔어.

User: 내 이름이 뭐라고 했지?
Assistant: 앨리스라고 했어.

User: 내가 어디 살고 있게?
Assistant: 시애틀이라고 했어.

User: 내가 몇 살 때 이사왔게?
Assistant: 21살 때 시애틀로 이사왔어.



### 구조화된 출력

이 예시는 모델이 정의된 스키마를 준수하는 구조화된 출력을 생성할 수 있는 방법을 보여줍니다

In [18]:
print(
    invoke("""
Write for each month name, the number of days in the that month and the holidays if any.
           
You MUST answer in JSON format only. Please follow the output schema below.
*Output Schema:*
[{
(month_name): {
    "number_of_days": (total number of days),
    "holidays": [( name each federal holiday )]
}
}]
"""
    )
)

[
  {
    "January": {
      "number_of_days": 31,
      "holidays": ["New Year's Day"]
    }
  },
  {
    "February": {
      "number_of_days": 28,
      "holidays": ["Washington's Birthday"]
    }
  },
  {
    "March": {
      "number_of_days": 31,
      "holidays": []
    }
  },
  {
    "April": {
      "number_of_days": 30,
      "holidays": ["Good Friday (not federal but observed)"]
    }
  },
  {
    "May": {
      "number_of_days": 31,
      "holidays": ["Memorial Day"]
    }
  },
  {
    "June": {
      "number_of_days": 30,
      "holidays": []
    }
  },
  {
    "July": {
      "number_of_days": 31,
      "holidays": ["Independence Day"]
    }
  },
  {
    "August": {
      "number_of_days": 31,
      "holidays": []
    }
  },
  {
    "September": {
      "number_of_days": 30,
      "holidays": ["Labor Day"]
    }
  },
  {
    "October": {
      "number_of_days": 31,
      "holidays": ["Columbus Day"]
    }
  },
  {
    "November": {
      "number_of_days": 30,
      "holiday

### 코드 설명 및 생성

다음 예시에서는 모델에게 코드를 생성하도록 요청합니다.

In [19]:
print(
    invoke(
        "Write a 'odd_even' function in Python that takes in a list of numbers, and return a tuple with (odd numbers, even numbers)",
    )
)

Certainly! Below is a Python function named `odd_even` that takes a list of numbers and returns a tuple containing two lists: one for odd numbers and one for even numbers.

```python
def odd_even(numbers):
    odd_numbers = []
    even_numbers = []
    
    for number in numbers:
        if number % 2 == 0:
            even_numbers.append(number)
        else:
            odd_numbers.append(number)
    
    return (odd_numbers, even_numbers)

# Example usage:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = odd_even(numbers)
print(result)  # Output: ([1, 3, 5, 7, 9], [2, 4, 6, 8, 10])
```

### Explanation:
- The function `odd_even` initializes two empty lists, `odd_numbers` and `even_numbers`.
- It then iterates over each number in the input list `numbers`.
- For each number, it checks if the number is even (`number % 2 == 0`). If it is, the number is appended to `even_numbers`. Otherwise, it is appended to `odd_numbers`.
- Finally, the function returns a tuple containing the two lis

### 분류

다음 예시는 분류를 위해 멀티턴과 함께 멀티모달 퓨샷 기능을 사용하는 방법을 보여줍니다.

In [20]:
from pathlib import Path

messages = [
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/cat.jpeg').read_bytes()}}}]},
    {"role": "assistant", "content": [{"text": "fail" }]},
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/dog.jpeg').read_bytes()}}}]},
    {"role": "assistant", "content": [{"text": "pass" }]},
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/dog2.jpeg').read_bytes()}}}]},
    {"role": "assistant", "content": [{"text": "pass" }]},
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/cat2.jpeg').read_bytes()}}}]},
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

response = client.converse(modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params)

response["output"]["message"]["content"][0]["text"]

'fail'